# Машинное обучение в бизнесе
# Домашняя работа к уроку № 6

In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [3]:
import numpy as np
import pandas as pd

import catboost as ctb

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [4]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """
    Обучить и оценить модель.
    """
    model = ctb.CatBoostClassifier(cat_features=cat_feats)
    model.fit(X_train, y_train, verbose=False)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='binary')
    rec = recall_score(y_test, y_pred, average='binary')
    
    return {'f1': [f1], 'roc-auc': [roc], 'precision': [prec], 'recall': [rec]}

## Задание 1
Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php).

Kaggle Dataset - HR Analytics: Job Change of Data Scientists:
- https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists

In [7]:
df = pd.read_csv("aug_train.csv")
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [8]:
df.shape

(19158, 14)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevent_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  target                  19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Ненужный признак

In [10]:
df = df.drop(columns=['enrollee_id'])

Конвертируем таргет в int

In [11]:
df['target'] = df['target'].astype(int)

Смотрим баланс таргета

In [12]:
df['target'].value_counts()

0    14381
1     4777
Name: target, dtype: int64

Просто заменяем все на самое частое значение (моду), т.к. все признаки с пропущенными значениями категориальные.

In [13]:
for col in df.select_dtypes('object').columns:
    df[col] = df[col].fillna(df[col].value_counts().index[0])

Делим данные на трейн и тест.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['target']), df['target'], test_size=0.2, random_state=42)

## Задание 2
Обучить любой классификатор (какой вам нравится).

In [15]:
cat_feats = ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level',
             'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']

In [16]:
metrics = pd.DataFrame(evaluate_model(ctb.CatBoostClassifier(cat_features=cat_feats),
                                      X_train,
                                      y_train,
                                      X_test,
                                      y_test))

In [17]:
metrics

,f1,roc-auc,precision,recall
0,0.503233,0.669061,0.571429,0.44958


## Задание 3
Далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть.

In [18]:
def create_unlabeled(df, pos_frac=0.2):
    """
    Сэмплирует долю pos_frac наблюдений класса 1 как positive, остальные как unlabeled.
    """
    sdf = df.copy()
    pos_mask = (df['target'] == 1)
    pos_ind = df[pos_mask].sample(frac=pos_frac).index
    unlab_ind = df[~df.index.isin(pos_ind)].index
    
    # Помечаем данные признаком is_labeled - Positive = 1, Unlabeled = 0
    df.loc[pos_ind, 'is_labeled'] = 1
    df.loc[unlab_ind, 'is_labeled'] = 0
    df['is_labeled'] = df['is_labeled'].astype(int)
    return df

Возьмем 20% наблюдений положительного класса как positive, остальные возьмем как unlabeled.

In [19]:
rns_df = create_unlabeled(df, pos_frac=0.2)

In [20]:
rns_df.head(3)

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,is_labeled
0,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,36,1,0
1,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0,0
2,city_21,0.624,Male,No relevent experience,Full time course,Graduate,STEM,5,50-99,Pvt Ltd,never,83,0,0


## Задание 4
Применить random negative sampling для построения классификатора в новых условиях.

In [21]:
def get_rns_samples(rns_df):
    """
    Создает тренировочную и тестовую выборки для RNS на основе признака is_labeled.
    """
    rns_df = rns_df.sample(frac=1)

    pos_sample = rns_df[rns_df['is_labeled'] == 1]
    neg_sample = rns_df[rns_df['is_labeled'] == 0][:pos_sample.shape[0]]
    train_samples = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    test_samples = rns_df[rns_df['is_labeled'] == 0][pos_sample.shape[0]:]
    
    return train_samples, test_samples

In [22]:
train_samples, test_samples = get_rns_samples(rns_df)

In [23]:
metrics_task5 = evaluate_model(ctb.CatBoostClassifier(cat_features=cat_feats),
                             train_samples.iloc[:, :-2],
                             train_samples['is_labeled'],
                             test_samples.iloc[:, :-2],
                             test_samples['target'])

In [24]:
metrics = metrics.append(pd.DataFrame(metrics_task5))

## Задание 5
Сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик).

In [25]:
metrics.index = ['normal', 'RNS']

In [26]:
metrics

,f1,roc-auc,precision,recall
normal,0.503233,0.669061,0.571429,0.449580
RNS,0.522379,0.727707,0.398686,0.757349


RNS справился даже немного лучше, чем обычная модель. Интересно то, что повысился recall и понизилась precision - модели стало сложнее различать между классами, поэтому она начала относить больше наблюдений к положительному классу.

## Задание 6*
Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P).

In [27]:
rns_metrics = pd.DataFrame(columns=['f1', 'roc-auc', 'precision', 'recall'])

fracs = np.linspace(0.1, 0.9, 9)
for frac in fracs:
    train_samples, test_samples = get_rns_samples(create_unlabeled(df, pos_frac=frac))
    frac_metrics = evaluate_model(ctb.CatBoostClassifier(cat_features=cat_feats),
                                 train_samples.iloc[:, :-2],
                                 train_samples['is_labeled'],
                                 test_samples.iloc[:, :-2],
                                 test_samples['target'])
    rns_metrics = rns_metrics.append(pd.DataFrame(frac_metrics))

In [28]:
rns_metrics.index = fracs

In [29]:
rns_metrics

,f1,roc-auc,precision,recall
0.1,0.532407,0.709522,0.435276,0.685340
0.2,0.531294,0.728231,0.419181,0.725275
0.3,0.503587,0.720916,0.401512,0.675253
0.4,0.471119,0.729997,0.346033,0.737834
0.5,0.447937,0.741437,0.320176,0.745357
0.6,0.415432,0.753966,0.285444,0.762808
0.7,0.345648,0.740011,0.226477,0.729515
0.8,0.270283,0.755941,0.163636,0.776078
0.9,0.163023,0.736954,0.091804,0.727019


По ROC-AUC, precision и recall лучшая доля сэмплинга - 0.6, по F1 score - 0.1.